# Mock Draft 2016

### Analyzing and modleing from the "expert" predictions

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from pandas import DataFrame
from sklearn import feature_selection

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Data is compiled in a .csv file containing first round draft predictions from ESPN (Todd McShay and/or Mel Kiper), Bleacher Report, and Walter Football.

#### First, which "expert" has been the most accurate over the past 10 years?

In [ ]:
with open("draft_mock_plus_stats.csv") as f:
    next(f)
    espn=0
    br=0
    wf=0
    total=0
    for line in f:
        if line.split(",")[6].strip() != "":
            total+=1
            if line.split(",")[2].strip() == line.split(",")[6].strip():
                espn+=1
            if line.split(",")[3].strip() == line.split(",")[6].strip():
                br+=1
            if line.split(",")[4].strip() == line.split(",")[6].strip():
                wf+=1
print "ESPN:\t\t", float(espn)/float(total)
print "BleacherReport:\t", float(br)/float(total)
print "WalterFootball:\t", float(wf)/float(total)

In [ ]:
pd.read_csv("draft_mock_plus_stats.csv").head()

Load in the relevant data and omit teams with no first round draft pick from the model:

In [ ]:
all_teams=pd.read_csv("draft_mock_plus_stats.csv")
picks=all_teams.dropna(subset=['IsOffense'])
print picks.head()

In [ ]:
df = pd.DataFrame(picks)
#print df
data_array = df.as_matrix()

In [ ]:
exp_preds = data_array[:, 2:5]
print exp_preds.shape
actual_pick = data_array[:,6]
exp_preds = pd.DataFrame(exp_preds, columns=['ESPN', 'BeacherReport', 'WalterFootball'])

print exp_preds.shape
print actual_pick.shape

X_train, X_test, y_train, y_test = cross_validation.train_test_split(exp_preds, actual_pick, test_size=0.25, random_state=3)

X_train.shape, y_train.shape


### Logistic Regression

In [ ]:
logreg = LogisticRegression()
scores=cross_val_score(logreg, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
print scores
print scores.mean()

In [ ]:
logreg_model = logreg.fit(X_train, y_train.astype(int))
print logreg_model.score(X_train, y_train.astype(int))
print logreg_model.predict([[0,0,1]])
#print logreg_model.predict([[0,1,0]])
#print logreg_model.predict([[0,0,1]])
#print logreg_model.predict([[0,1,1]])

### Random Forest


In [ ]:
rf_model = RandomForestClassifier()
scores=cross_val_score(rf_model, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
#rf_model = rf_model.fit(X_train, y_train)
print scores
print scores.mean()

### Support Vector Machine

In [ ]:
svm_model=svm.SVC()
scores=cross_val_score(svm_model, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
print scores
print scores.mean()

In [ ]:
feature_selection.f_classif(exp_preds, np.array(actual_pick.astype(int)))

### K-Nearest Neighbors

In [ ]:
k_range=range(1,51)
k_scores1=[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
    k_scores1.append(scores.mean())
print k_scores1
avg_kscore = sum(k_scores1)/len(k_scores1)


In [ ]:
plt.plot(k_range, k_scores1)
plt.show()

#### Max accuracy at k=13

In [ ]:
feature_selection.f_classif(exp_preds, np.array(actual_pick.astype(int)))

## Now what happens to accuracy when we also consider stats from previous season?

In [ ]:
exp_preds = data_array[:, 2:6]
actual_pick = data_array[:,6]
exp_preds = pd.DataFrame(exp_preds, columns=['ESPN', 'BeacherReport', 'WalterFootball', 'OffensiveBias'])


print exp_preds.shape
print actual_pick.shape


X_train, X_test, y_train, y_test = cross_validation.train_test_split(exp_preds, actual_pick, test_size=0.25, random_state=3)

X_train.shape, y_train.shape

### Logistic Regression

In [ ]:
logreg = LogisticRegression()
scores=cross_val_score(logreg, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
print scores
print scores.mean()

In [ ]:
logreg_model = logreg.fit(X_train, y_train.astype(int))
print logreg_model.score(X_train, y_train.astype(int))
print logreg_model.predict([[0,1,1,32]])


### Random Forest

In [ ]:
rf_model = RandomForestClassifier()
scores=cross_val_score(rf_model, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
print scores
print scores.mean()

### Support Vector Machine

In [ ]:
svm_model=svm.SVC()
scores=cross_val_score(svm_model, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
print scores
print scores.mean()

### K-Nearest Neighbors


In [ ]:
k_range=range(1,51)
k_scores=[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn, exp_preds, np.array(actual_pick.astype(int)), cv=10, scoring='accuracy')
    k_scores.append(scores.mean())
print k_scores
avg_kscore = sum(k_scores)/len(k_scores)


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(k_range, k_scores1, label='Without Team Stats')
plt.plot(k_range, k_scores, label='With Team Stats')
plt.xticks(fontsize=14)
plt.xlabel('Neighbors', fontsize='28')
plt.yticks(fontsize=14)
plt.ylabel('Accuracy', fontsize='28')
legend = plt.legend(loc='center right', prop={'size':16})
plt.show()

In [ ]:
feature_selection.f_classif(exp_preds, np.array(actual_pick.astype(int)))